<a href="https://colab.research.google.com/github/anash18/4105-HW-Nash/blob/main/ECGR_4105_Final_Project_Pytorch/Pytorch/Pytorch_Heart_Attack_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.63 s (started: 2023-12-15 23:07:20 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 782 ms (started: 2023-12-15 23:07:24 +00:00)


In [ ]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/Data/heart_attack_prediction_dataset_1.csv'

heart_attack = pd.read_csv(file_path)

heart_attack.head()

,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,67,Male,208,158,72,0,0,1,0,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,21,Male,389,165,98,1,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,21,Female,324,174,72,1,0,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,84,Male,383,163,73,1,1,1,0,1,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,66,Male,318,91,93,1,1,1,1,0,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


time: 71.7 ms (started: 2023-12-15 23:07:25 +00:00)


In [ ]:
#Defining mapping function to map multiple inputs of the dataset
def gender_map(x):
    return x.map({'M': 1, 'F': 0, 'Male' : 1, "Female" : 0, '1' : 1, '0' : 0})

def hemisphere_map(x):
    return x.map({'Southern Hemisphere' : -1, 'Northern Hemisphere' : 1})

def countries_map(x):
    return x.map({'Argentina' : -1, 'Brazil' : -1, 'China' : -1, 'Colombia' : -1,
    'India' : -1, 'Nigeria' : -1, 'South Africa' : -1, 'South Korea' : -1, 'Thailand' : -1,
    'Vietnam' : -1, 'Australia' : 1, 'Canada' : 1, 'France' : 1, 'Germany' : 1,
    'Italy' : 1, 'Japan' : 1, 'New Zealand' : 1, 'Spain' : 1, 'United Kingdom' : 1,
    'United States' : 1})

def diet_map(x):
    return x.map({'Unhealthy' : -1, 'Average' : 0, 'Healthy' : 1})

def continent_map(x):
    return x.map({'Asia' : 0, 'Africa' : 1, 'Europe' : 2, 'North America' : 3,
    'South America' : 4, 'Australia' : 5})

time: 1.11 ms (started: 2023-12-15 23:07:25 +00:00)


In [ ]:
heart_attack['Sex'] = heart_attack[['Sex']].apply(gender_map)
heart_attack['Hemisphere'] = heart_attack[['Hemisphere']].apply(hemisphere_map)
heart_attack['Country'] = heart_attack[['Country']].apply(countries_map)
heart_attack['Diet'] = heart_attack[['Diet']].apply(diet_map)
heart_attack['Continent'] = heart_attack[['Continent']].apply(continent_map)

heart_attack.head()

,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,67,1,208,158,72,0,0,1,0,0,...,6.615001,261404,31.251233,286,0,6,-1,4,-1,0
1,21,1,389,165,98,1,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,1,3,1,0
2,21,0,324,174,72,1,0,0,0,0,...,9.463426,235282,28.176571,587,4,4,1,2,1,0
3,84,1,383,163,73,1,1,1,0,1,...,7.648981,125640,36.464704,378,3,4,1,3,1,0
4,66,1,318,91,93,1,1,1,1,0,...,1.514821,160555,21.809144,231,1,5,-1,0,1,0


time: 47.9 ms (started: 2023-12-15 23:07:25 +00:00)


In [ ]:
#Splitting the Variables into X and Y
X = heart_attack.iloc[:, 0:23].values
Y = heart_attack.iloc[:, 24].values

# Split our Data set into Training Data and val Data.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# Use standard scaling from Sklearn to scale data between - and  for better accuracy.
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_val = scale.transform(X_val)

Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)

X_train_t = torch.tensor(X_train, dtype=torch.float)
X_val_t = torch.tensor(X_val, dtype=torch.float)
Y_train_t = torch.tensor(Y_train, dtype=torch.float)
Y_val_t = torch.tensor(Y_val, dtype=torch.float)


print(X_train_t.dtype)
print(X_val_t.dtype)
print(Y_train_t.dtype)
print(Y_val_t.dtype)

torch.float32
torch.float32
torch.float32
torch.float32
time: 18.9 ms (started: 2023-12-15 23:07:25 +00:00)


In [ ]:
def model_linear(t_in, w1, w2, w3, b):
  t_p = ((w3 * t_in ** 3) + (w2 * t_in ** 2) + (w1 * t_in) + b)
  return t_p

def loss_fn(t_p, t_gnd):
  squared_diffs = (t_p - t_gnd)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_in, val_t_in,
                    train_t_out, val_t_out):
  for epoch in range(1, n_epochs + 1):

    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model_linear(train_t_in, *params)
    train_loss = loss_fn(train_t_p, train_t_out)

    val_t_p = model_linear(val_t_in, *params)
    val_loss = loss_fn(val_t_p, val_t_out)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
      f" \tValidation loss {val_loss.item():.4f}")

  return params

time: 5.47 ms (started: 2023-12-15 23:07:25 +00:00)


In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 0.0], requires_grad=True)

learning_rate_1 = 1e-3
optimizer_1 = optim.SGD([params], lr=learning_rate_1)

print('Loss every 500 epochs using learning rate of 1e-3\n')
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1,
    params = params,
    train_t_in = X_train_t,
    val_t_in = X_val_t,
    train_t_out = Y_train_t,
    val_t_out = Y_val_t)

Loss every 500 epochs using learning rate of 1e-3

Epoch 1, 	Training loss 9.4821, 	Validation loss 9.6949
Epoch 2, 	Training loss 9.2767, 	Validation loss 9.4803
Epoch 3, 	Training loss 9.0767, 	Validation loss 9.2713
Epoch 500, 	Training loss 0.4707, 	Validation loss 0.4776
Epoch 1000, 	Training loss 0.3346, 	Validation loss 0.3399
Epoch 1500, 	Training loss 0.2806, 	Validation loss 0.2845
Epoch 2000, 	Training loss 0.2551, 	Validation loss 0.2582
Epoch 2500, 	Training loss 0.2426, 	Validation loss 0.2454
Epoch 3000, 	Training loss 0.2364, 	Validation loss 0.2390
Epoch 3500, 	Training loss 0.2332, 	Validation loss 0.2357
Epoch 4000, 	Training loss 0.2315, 	Validation loss 0.2339
Epoch 4500, 	Training loss 0.2306, 	Validation loss 0.2330
Epoch 5000, 	Training loss 0.2301, 	Validation loss 0.2325


tensor([-0.0090,  0.0318,  0.0083,  0.3154], requires_grad=True)

time: 25 s (started: 2023-12-15 23:07:25 +00:00)


In [ ]:
# Getting the predictions based on the training set using the linear model
model_output = model_linear(X_train_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_np = model_output.detach().numpy() if isinstance(model_output, torch.Tensor) else model_output

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels = np.argmax(model_output_np, axis=1)

# Calculate accuracy, precision, and recall for training set
train_accuracy = accuracy_score(Y_train, predicted_labels)
train_recall = recall_score(Y_train, predicted_labels, average= 'macro', zero_division=1)
train_precision = precision_score(Y_train, predicted_labels, average = 'macro', zero_division=1)
train_F1_score = (2*train_recall*train_precision)/(train_recall+train_precision)

# Getting the predictions based on the validation set using the linear model
model_output_val = model_linear(X_val_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_val_np = model_output_val.detach().numpy() if isinstance(model_output_val, torch.Tensor) else model_output_val

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels_val = np.argmax(model_output_val_np, axis=1)

# Calculate accuracy, precision, and recall for validation set
val_accuracy = accuracy_score(Y_val, predicted_labels_val)
val_precision = precision_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_recall = recall_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_F1_score = (2*val_recall*val_precision)/(val_recall+val_precision)

# Print the calculated metrics
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Training F1 Score: {train_F1_score:.4f}")

print(f"\nValidation Accuracy: {val_accuracy:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Validation F1 Score: {val_F1_score:.4f}")

Training Accuracy: 0.0516
Training Precision: 0.0470
Training Recall: 0.9044
Training F1 Score: 0.0893

Validation Accuracy: 0.0525
Validation Precision: 0.0615
Validation Recall: 0.8938
Validation F1 Score: 0.1151
time: 18.6 ms (started: 2023-12-15 23:07:50 +00:00)


In [ ]:
#Sequential Model

model_seq = nn.Sequential(
            nn.Linear(23, 1024),
            nn.Tanh(),
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Linear(2048, 512),
            nn.Tanh(),
            nn.Linear(512,1)
            )
model_seq

Sequential(
  (0): Linear(in_features=23, out_features=1024, bias=True)
  (1): Tanh()
  (2): Linear(in_features=1024, out_features=2048, bias=True)
  (3): Tanh()
  (4): Linear(in_features=2048, out_features=512, bias=True)
  (5): Tanh()
  (6): Linear(in_features=512, out_features=1, bias=True)
)

time: 37 ms (started: 2023-12-15 23:07:50 +00:00)


In [ ]:
# Checking the complexity of the model
sum([p.numel() for p in model_seq.parameters()])

3173377

time: 4.56 ms (started: 2023-12-15 23:07:51 +00:00)


In [ ]:
#Let's define our training loop that will be used

def training_loop(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()


        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

time: 478 µs (started: 2023-12-15 23:07:51 +00:00)


In [ ]:
#Let's try doing some training

optimizer_1 = optim.SGD(model_seq.parameters(), lr=1e-2)

training_loop(
    n_epochs = 501,
    optimizer = optimizer_1,
    model = model_seq,
    loss_fn = nn.MSELoss(),
    x_train = X_train_t,
    x_val = X_val_t,
    y_train = Y_train_t,
    y_val = Y_val_t
)

Epoch: 1, Training loss: 0.3883, Validation loss: 0.3935
Epoch: 10, Training loss: 0.2585, Validation loss: 0.2634
Epoch: 20, Training loss: 0.2334, Validation loss: 0.2376
Epoch: 30, Training loss: 0.2296, Validation loss: 0.2335
Epoch: 40, Training loss: 0.2289, Validation loss: 0.2328
Epoch: 50, Training loss: 0.2288, Validation loss: 0.2327
Epoch: 60, Training loss: 0.2287, Validation loss: 0.2327
Epoch: 70, Training loss: 0.2287, Validation loss: 0.2327
Epoch: 80, Training loss: 0.2286, Validation loss: 0.2327
Epoch: 90, Training loss: 0.2286, Validation loss: 0.2328
Epoch: 100, Training loss: 0.2286, Validation loss: 0.2328
Epoch: 110, Training loss: 0.2285, Validation loss: 0.2328
Epoch: 120, Training loss: 0.2285, Validation loss: 0.2328
Epoch: 130, Training loss: 0.2285, Validation loss: 0.2328
Epoch: 140, Training loss: 0.2285, Validation loss: 0.2328
Epoch: 150, Training loss: 0.2284, Validation loss: 0.2328
Epoch: 160, Training loss: 0.2284, Validation loss: 0.2329
Epoch: 1

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(predictions, targets):
    return (predictions == targets).float().mean().item()

# Function to calculate recall
def calculate_recall(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_negatives = ((predictions == 0) & (targets == 1)).sum().item()
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Function to calculate precision
def calculate_precision(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_positive = ((predictions == 1) & (targets == 0)).sum().item()
    return true_positives / (true_positives + false_positive) if (true_positives + false_positive) > 0 else 0


time: 706 µs (started: 2023-12-15 23:20:03 +00:00)


In [ ]:
# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_train_t)))

# Calculate accuracy, recall, precision, and F1 score for the training set
accuracy_train = calculate_accuracy(train_predictions, Y_train_t)
recall_train = calculate_recall(train_predictions, Y_train_t)
precision_train = calculate_precision(train_predictions, Y_train_t)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_val_t)))

# Calculate accuracy, recall, precision, and F1 score for the validation set
accuracy_val = calculate_accuracy(train_predictions, Y_val_t)
recall_val = calculate_recall(train_predictions, Y_val_t)
precision_val = calculate_precision(train_predictions, Y_val_t)
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)


# Print the metrics
print("Training Accuracy: {:.4f}".format(accuracy_train))
print("Training Recall: {:.4f}".format(recall_train))
print("Training Precision: {:.4f}".format(precision_train))
print("Training F1 Score: {:.4f}".format(F1_score_train))

print("\nValidation Accuracy: {:.4f}".format(accuracy_val))
print("Validation Recall: {:.4f}".format(recall_val))
print("Validation Precision: {:.4f}".format(precision_val))
print("Validation F1 Score: {:.4f}".format(F1_score_val))

Training Accuracy: 0.3566
Training Recall: 1.0000
Training Precision: 0.3566
Training F1 Score: 0.5258

Validation Accuracy: 0.3645
Validation Recall: 1.0000
Validation Precision: 0.3645
Validation F1 Score: 0.5343
time: 522 ms (started: 2023-12-15 23:20:03 +00:00)


In [ ]:
# Defining the fully connected neural network

model_fnn = nn.Sequential(
            nn.Linear(23, 1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(2048,1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(512, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
           )

time: 34.3 ms (started: 2023-12-15 23:20:04 +00:00)


In [ ]:
# Checking the complexity of the model
sum([p.numel() for p in model_fnn.parameters()])

4779649

time: 2.98 ms (started: 2023-12-15 23:20:04 +00:00)


In [ ]:
# Setting up the parameters used in the training loop

learning_rate = 1

optimizer = optim.SGD(model_fnn.parameters(), lr=learning_rate)

loss_fn = nn.BCEWithLogitsLoss()

n_epochs = 501

# This is the Training

for epoch in range(n_epochs):
    outputs = model_fnn(X_train_t)
    loss = loss_fn(outputs, Y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # Print loss every 10 epochs
      print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.700768
Epoch: 10, Loss: 0.650140
Epoch: 20, Loss: 0.651026
Epoch: 30, Loss: 0.651592
Epoch: 40, Loss: 0.650889
Epoch: 50, Loss: 0.650021
Epoch: 60, Loss: 0.650214
Epoch: 70, Loss: 0.650117
Epoch: 80, Loss: 0.649666
Epoch: 90, Loss: 0.650781
Epoch: 100, Loss: 0.649681
Epoch: 110, Loss: 0.650135
Epoch: 120, Loss: 0.649348
Epoch: 130, Loss: 0.650342
Epoch: 140, Loss: 0.649816
Epoch: 150, Loss: 0.650650
Epoch: 160, Loss: 0.650199
Epoch: 170, Loss: 0.650165
Epoch: 180, Loss: 0.650469
Epoch: 190, Loss: 0.650063
Epoch: 200, Loss: 0.650060
Epoch: 210, Loss: 0.649326
Epoch: 220, Loss: 0.649404
Epoch: 230, Loss: 0.649868
Epoch: 240, Loss: 0.650637
Epoch: 250, Loss: 0.649840
Epoch: 260, Loss: 0.649746
Epoch: 270, Loss: 0.649233
Epoch: 280, Loss: 0.649198
Epoch: 290, Loss: 0.650148
Epoch: 300, Loss: 0.650180
Epoch: 310, Loss: 0.649364
Epoch: 320, Loss: 0.649414
Epoch: 330, Loss: 0.649563
Epoch: 340, Loss: 0.649506
Epoch: 350, Loss: 0.650425
Epoch: 360, Loss: 0.649664
Epoch: 370, 

In [ ]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Training Set

correct_train = 0
total_train = 0

with torch.no_grad():
    outputs_train = model_fnn(X_train_t)
    _, predicted_train = torch.max(outputs_train, dim=1)
    total_train += Y_train_t.size(0)
    correct_train += int((predicted_train == Y_train_t.squeeze()).sum())

accuracy_train = correct_train / total_train
precision_train = precision_score(Y_train_t, predicted_train, average='macro', zero_division=0)
recall_train = recall_score(Y_train_t, predicted_train, average='macro', zero_division=0)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Print the metrics
print("Training Accuracy: %f" % accuracy_train)
print("Training Precision: %f" % precision_train)
print("Training Recall: %f" % recall_train)
print("Training F1 score: %f" % F1_score_train)

Training Accuracy: 0.643367
Training Precision: 0.321683
Training Recall: 0.500000
Training F1 score: 0.391493
time: 757 ms (started: 2023-12-15 23:36:57 +00:00)


In [ ]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Validation Set

correct_val = 0
total_val = 0

with torch.no_grad():
    outputs_val = model_fnn(X_val_t)
    _, predicted_val = torch.max(outputs_val, dim=1)
    total_val += Y_val_t.size(0)
    correct_val += int((predicted_val == Y_val_t.squeeze()).sum())

accuracy_val = correct_val / total_val
precision_val = precision_score(Y_val_t, predicted_val, average='macro', zero_division=0)
pecall_val = recall_score(Y_val_t, predicted_val, average='macro', zero_division=0)
p1_score_train = (2*recall_val*precision_val)/(recall_val+precision_val)

# Print the metrics
print("Validation Accuracy: %f" % accuracy_val)
print("Validation Precision: %f" % precision_val)
print("Validation Recall: %f" % recall_val)
print("Validation F1 score: %f" % F1_score_train)

Validation Accuracy: 0.635482
Validation Precision: 0.317741
Validation Recall: 1.000000
Validation F1 score: 0.391493
time: 173 ms (started: 2023-12-15 23:36:57 +00:00)
